<a href="https://colab.research.google.com/github/BANZOM/MachingLearningStuffs/blob/main/optimizationOfCNNmoDel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.8 MB/s 
     |████████████████████████████████| 1.6 MB 40.3 MB/s 


In [2]:
import tensorflow as ts
from tensorflow import keras
import numpy as np


In [3]:
# importing the model
Fashion_mnist= keras.datasets.fashion_mnist

In [4]:
# Partitioning of dataset
(train_images,train_label),(test_images,test_label) = Fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# as its a gray scale image
# scaling the data in 0 to 1 so to train
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
# checking the dimention of the data
train_images[0].shape

(28, 28)

In [7]:
# changing the dimensions so to include all the data 
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
# Rechecking the dimentions
train_images[0].shape

(28, 28, 1)

In [9]:
# function inspired from the keras library

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=16, max_value=256, step=8),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=16, max_value=256, step=8),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=16, max_value=256, step=8),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
# additonal library for finding the best possible filters
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
# searcinh the best
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_label,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9056666493415833

Best val_accuracy So Far: 0.9114999771118164
Total elapsed time: 00h 04m 46s


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 168)       1680      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 136)       205768    
                                                                 
 flatten (Flatten)           (None, 78336)             0         
                                                                 
 dense (Dense)               (None, 120)               9400440   
                                                                 
 dense_1 (Dense)             (None, 10)                1210      
                                                                 
Total params: 9,609,098
Trainable params: 9,609,098
Non-trainable params: 0
_________________________________________________________________


In [23]:

model.fit(train_images, train_label, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0954 - accuracy: 0.9648 - val_loss: 0.2722 - val_accuracy: 0.9163
Epoch 5/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0616 - accuracy: 0.9772 - val_loss: 0.3512 - val_accuracy: 0.9133
Epoch 6/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0417 - accuracy: 0.9848 - val_loss: 0.3796 - val_accuracy: 0.9177
Epoch 7/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0334 - accuracy: 0.9882 - val_loss: 0.4805 - val_accuracy: 0.9100
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0219 - accuracy: 0.9924 - val_loss: 0.4582 - val_accuracy: 0.9117
Epoch 9/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0195 - accuracy: 0.9931 - val_loss: 0.5487 - val_accuracy: 0.9120
Epoch 10/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0166 - accuracy: 0.9945 - val_loss: 0.5586 

In [25]:
model.fit(train_images, train_label, epochs=20, validation_split=0.1, initial_epoch=10)


Epoch 11/20
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0165 - accuracy: 0.9946 - val_loss: 0.5977 - val_accuracy: 0.9162
Epoch 12/20
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0149 - accuracy: 0.9950 - val_loss: 0.6737 - val_accuracy: 0.9048
Epoch 13/20
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0129 - accuracy: 0.9955 - val_loss: 0.7630 - val_accuracy: 0.9093
Epoch 14/20
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0116 - accuracy: 0.9962 - val_loss: 0.9277 - val_accuracy: 0.9052
Epoch 15/20
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0129 - accuracy: 0.9959 - val_loss: 0.8409 - val_accuracy: 0.9100
Epoch 16/20
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.8200 - val_accuracy: 0.9110
Epoch 17/20
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0110 - accuracy: 0.9967 - val_loss: 0